In [1]:
import numpy as np
import scipy.optimize as op
import scipy.stats as ss
import matplotlib.pyplot as plt
import ARTsampler
import emcee
import corner

%matplotlib inline
np.random.seed(42)

In [2]:
# a posterior
_variances = np.array([0.1, 2.])**2
_corr = 0.3
_Sigma = np.diag(_variances)
_Sigma[0,1] = _Sigma[1,0] = _corr*np.sqrt(np.prod(_Sigma.diagonal()))
_w = 0.9
_F1 = ss.multivariate_normal(mean=np.array([0.1, 4]), cov=_Sigma)
_F2 = ss.multivariate_normal(mean=np.array([0.4, 4]), cov=_Sigma)
print(_Sigma)

def lnpost(x):
    return np.log(_w*_F1.pdf(x) + (1-_w)*_F2.pdf(x)) + 10 #plus a DC offset for testing
    
def neg_lnpost(x):
    return -lnpost(x)

[[0.01 0.06]
 [0.06 4.  ]]


In [3]:
sampler = ARTsampler.ARTsampler(lnpost, lnprob_args=(), x0=[0,0], 
                                temperature=20, multiplicity = 30,
                                quiet=False, verbose=True)

Running the minimizer
      fun: -9.714149417629491
 hess_inv: array([[0.01004116, 0.05999296],
       [0.05999296, 3.99999787]])
      jac: array([-6.91413879e-06,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 32
      nit: 4
     njev: 8
   status: 0
  success: True
        x: array([0.10023889, 3.99999964])


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


TypeError: putmask() argument 1 must be numpy.ndarray, not numpy.float64

In [ ]:
pos = np.array([0.3, 3]) + 1e-2 * np.random.randn(32, 2)
nwalkers, ndim = pos.shape

Esampler = emcee.EnsembleSampler(nwalkers, ndim, sampler.logpdf)
Esampler.run_mcmc(pos, 6000)
samples = Esampler.get_chain(flat=True)

In [ ]:
plt.scatter(samples[:,0], samples[:,1], c='k', alpha=0.2, s=0.2)
plt.scatter(sampler.g[:,0], sampler.g[:,1], c="blue", s=1)

In [ ]:
xs = np.linspace(-.5, 1)
xs = np.vstack((xs,xs)).T
xs[:,1] = 7
lnG = sampler.Gaussian_component_logpdf(xs)
lnp = sampler.logpdf(xs)
TRUTH = np.array([lnpost(xi) for xi in xs])
plt.plot(xs[:,0], lnp)
plt.plot(xs[:,0], lnG)
plt.plot(xs[:,0], TRUTH, "--")
plt.ylim(min(TRUTH), max(TRUTH)*2)
plt.axvline(0.3, c='k')
plt.axvline(0.3 + 5*0.13, c="k", ls="--")
plt.axvline(0.3 - 5*0.13, c="k", ls="--")
plt.axvline(0.3 + 3*0.13, c="r", ls="--")
plt.axvline(0.3 - 3*0.13, c="r", ls="--")

In [ ]:
xs = np.linspace(-16, 20)
xs = np.vstack((xs,xs)).T
xs[:,0] = 0.3
lnG = sampler.Gaussian_component_logpdf(xs)
lnp = sampler.logpdf(xs)
TRUTH = np.array([lnpost(xi) for xi in xs])
plt.plot(xs[:,1], lnp)
plt.plot(xs[:,1], lnG)
plt.plot(xs[:,1], TRUTH, "--")
plt.ylim(-20, max(TRUTH))
plt.axvline(4, c='k')
plt.axvline(4 + 5*2, c="k", ls="--")
plt.axvline(4 - 5*2, c="k", ls="--")
plt.axvline(4 + 3*2, c="r", ls="--")
plt.axvline(4 - 3*2, c="r", ls="--")

In [ ]:
fig = corner.corner(samples)

In [ ]:
if True:
    Esampler2 = emcee.EnsembleSampler(nwalkers, ndim, lnpost)
    Esampler2.run_mcmc(pos, 3000)
    samples = Esampler2.get_chain(flat=True)
    fig = corner.corner(samples)